In [ ]:
from src.llm_plan.llm import GPT_OSS

system_prompt = "You are a useful assistant."
prompt = "Tell me a joke."

model = GPT_OSS(model_name="openai/gpt-oss-120b", map="cpu", reasoning="Low")

model.generate_sync(system_prompt, prompt)

In [ ]:
prompt = "What is the capital of France?"
model.generate_sync(system_prompt, prompt)